# Laboratorio de Introducción al Procesamiento de Lenguaje Natural 2016

## Importación de las sentencias del corpus del Poder Judicial.

In [1]:
import pandas

corpus = pandas.read_csv("corpus_pj.csv", delimiter=',', skip_blank_lines=True, encoding='utf-8')
corpus = corpus.drop_duplicates()
sentencias = corpus["sentencia"]


## Separar en oraciones y tokens.

In [2]:
import nltk

oraciones = []

sent_detector = nltk.data.load('tokenizers/punkt/spanish.pickle')

for sentencia in sentencias:
    for oracion in sent_detector.tokenize(sentencia):
        oraciones.append(oracion)
        # oraciones.append(nltk.word_tokenize(oracion))
              
    

## Analizar tokens.

In [6]:
# -*- encoding: utf-8 -*-

import sys
import json

sys.path.append('.../myfreeling/APIs/python')
import freeling
import codecs
import re

# Configuracion de freeling
# -------------------------
FREELINGDIR = "/usr/local";
DATA = FREELINGDIR+"/share/freeling/"
LANG = "es"

freeling.util_init_locale("default")

op = freeling.maco_options("es")
# # op.set_active_modules(1,1,1,1,1,1,1,1,1,1,0)
op.set_data_files( "", 
                   DATA + "common/punct.dat",
                   DATA + LANG + "/dicc.src",
                   DATA + LANG + "/afixos.dat",
                   "",
                   DATA + LANG + "/locucions.dat", 
                   DATA + LANG + "/np.dat",
                   DATA + LANG + "/quantities.dat",
                   DATA + LANG + "/probabilitats.dat")
op.set_retok_contractions(False)

# lg  = freeling.lang_ident(DATA+"common/lang_ident/ident-few.dat")
mf  = freeling.maco(op)

tk  = freeling.tokenizer(DATA+LANG+"/tokenizer.dat")
sp  = freeling.splitter(DATA+LANG+"/splitter.dat")
tg  = freeling.hmm_tagger(DATA+LANG+"/tagger.dat",1,2)

# sen = freeling.senses(DATA+LANG+"/senses.dat");
# ukb = freeling.ukb(DATA+LANG+"/ukb.dat")

def tag (sent):
    out = {}
    lang = "es"
    l = tk.tokenize(sent)
    ls = sp.split(l) # old value 0
    ls = mf.analyze(ls)
    ls = tg.analyze(ls)
    wss = []
    for s in ls:
        ws = s.get_words()
        for w in ws:
            an = w.get_analysis()
            a = an[0]
            wse = dict(wordform =  w.get_form(),
                       lemma = a.get_lemma(),
                       tag = a.get_tag(),
                       prob = a.get_prob(),
                       analysis = len(an))
            wss.append(wse)
    out['words'] = wss
    out['lang'] = lang
    return out

corpus_taggeado = []
for s in oraciones[1:3]:
    corpus_taggeado.append(tag(s))

print len(corpus_taggeado)
print corpus_taggeado[0]

2
{'lang': 'es', 'words': [{'lemma': u'resultando_que', 'wordform': u'RESULTANDO_QUE', 'tag': u'NP00000', 'prob': 1.0, 'analysis': 1}, {'lemma': u':', 'wordform': u':', 'tag': u'Fd', 'prob': 1.0, 'analysis': 1}, {'lemma': u'1', 'wordform': u'1', 'tag': u'Z', 'prob': 1.0, 'analysis': 1}, {'lemma': u'por', 'wordform': u'Por', 'tag': u'NP00000', 'prob': 1.0, 'analysis': 1}, {'lemma': u'sentencia', 'wordform': u'sentencia', 'tag': u'NCFS000', 'prob': 0.9722863741339492, 'analysis': 3}, {'lemma': u'no', 'wordform': u'No', 'tag': u'NP00000', 'prob': 1.0, 'analysis': 1}, {'lemma': u'.', 'wordform': u'.', 'tag': u'Fp', 'prob': 1.0, 'analysis': 1}]}


## Identificar entidades con nombre.